In [13]:
#https://www.kaggle.com/datasets/kazanova/sentiment140 for the dataset.
#check for even distribution of targets, number of null values FOR ALL ML DATASETS.

import csv
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ishaa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
print(stopwords.words('english'))
#stopwords are those words which do not influence the data. They are not required for processing.

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [15]:
#processing the dataset
#open the csv dataset using pandas
twitter_data = pd.read_csv("archive\dataset.csv", encoding = 'ISO-8859-1')
#check the number of rows and columns:
twitter_data.shape
#print the first 5 rows of df:
twitter_data.head()

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\ishaa\AppData\Local\Temp\ipykernel_10676\2522207774.py:3: SyntaxWarning: invalid escape sequence '\d'
  twitter_data = pd.read_csv("archive\dataset.csv", encoding = 'ISO-8859-1')


,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [16]:
#naming the columns and then rereading the data: (as the csv file is not reading the dataset column names.)
column_names = ['target', 'id', 'data', 'flag', 'user', 'text'] #target ranges from 0(negative) to 4(positive)
twitter_data = pd.read_csv("archive\dataset.csv", names = column_names, encoding = 'ISO-8859-1')
twitter_data.head()

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\ishaa\AppData\Local\Temp\ipykernel_10676\2390823295.py:3: SyntaxWarning: invalid escape sequence '\d'
  twitter_data = pd.read_csv("archive\dataset.csv", names = column_names, encoding = 'ISO-8859-1')


,target,id,data,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [17]:
#counting the number of missing values / text in the dataset: 
twitter_data.isnull().sum()

target    0
id        0
data      0
flag      0
user      0
text      0
dtype: int64

In [18]:
#checking the distribution of positive and negative tweets (from the target columns)
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [19]:
#change the range to make it 0-1 range rather than 0-4 range:
twitter_data.replace({'target':{4:1}}, inplace=True)
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [20]:
#Stemming process using Porterstemmer to reduce words to their roots.
#try lemmatization as well later maybe?

portstem = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content) #remove all charcters which are not alphabets.
    stemmed_content = stemmed_content.lower()   #convert to lower-case
    stemmed_content = stemmed_content.split()   #seperate the words out.
    stemmed_content = [portstem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]   #stem the words which are not in stopwords.
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content
twitter_data['cleaned_content'] = twitter_data['text'].apply(stemming)

#this cell took 448 minutes, 50 seconds to train.

In [21]:
twitter_data.head()  

,target,id,data,flag,user,text,cleaned_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [22]:
#seperating the data and the label
X = twitter_data['cleaned_content'].values
Y = twitter_data['target'].values
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [23]:
print(Y)

[0 0 0 ... 1 1 1]


In [34]:
#splitting the data - train-test model
X_train, X_test, Y_train, Y_test=train_test_split(X, Y, test_size=0.2, stratify=Y) 
#stratify means I want even split of training and test data classes - all negative values shdnt end up only in training.
#random state=2 sets a random state so that everytime the data is split the same - to get same values while following a tutorial or something
print(X.shape, X_train.shape, X_test.shape)
print(Y.shape, Y_train.shape, Y_test.shape)



(1600000,) (1280000,) (320000,)
(1600000,) (1280000,) (320000,)


In [25]:
#Feature extraction - converting textual data to numerical data
vectorizer = TfidfVectorizer()      #converts all words to some numerical values based on significance
X_train = vectorizer.fit_transform(X_train)     #transforming the training data using the vectorizer - tries to understand what is the nature of the data
X_test = vectorizer.transform(X_test)           #transforms the data, but does not understand the nature of the data.

In [26]:
print(X_train)
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9453159 stored elements and shape (1280000, 461351)>
  Coords	Values
  (0, 314667)	0.6343358422454248
  (0, 321914)	0.5133215435244839
  (0, 354636)	0.3715154090741053
  (0, 165545)	0.4428265270023116
  (1, 263430)	0.7715756395081228
  (1, 443567)	0.2607619154161615
  (1, 63651)	0.34133239787474745
  (1, 384543)	0.24809594508684035
  (1, 440796)	0.3982648015813957
  (2, 253311)	0.5158455059443257
  (2, 89269)	0.21265450151068035
  (2, 316102)	0.37613176930350073
  (2, 138780)	0.18488593609381113
  (2, 136681)	0.22365553110037342
  (2, 250711)	0.22284737355265008
  (2, 100637)	0.25908385437957504
  (2, 437089)	0.18556308172176902
  (2, 145820)	0.13326127160264425
  (2, 340709)	0.30591043766146664
  (2, 124303)	0.1888295982474079
  (2, 249121)	0.16639494371709498
  (2, 240524)	0.1693668017700601
  (2, 234589)	0.14329188601587275
  (2, 2492)	0.2964479302713676
  (3, 349600)	0.40962607920923105
  :	:
  (1279996, 354532)	0.194681

In [27]:
#Training the model using logistic regression:

model = LogisticRegression(max_iter=50000)   #50000 iterations
model.fit(X_train, Y_train)

LogisticRegression(max_iter=50000)

In [28]:
#model evaluation: checks how many predictions are right from training data
X_train_prediction = model.predict(X_train)     #predict target values for training data tweets
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)    #compare the predictions with the actual values
print('accuracy_score on training data:', training_data_accuracy)



accuracy_score on training data: 0.7987984375


In [29]:
X_test_prediction = model.predict(X_test)     
testing_data_accuracy = accuracy_score(Y_test, X_test_prediction)    
print('accuracy_score on testing data:', testing_data_accuracy)

accuracy_score on testing data: 0.777684375


#Accuracy Score = 77.78%

In [30]:
import pickle
#saving the trained model
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))    #model is the name of the model we are saving


In [32]:
#using the saved model for future predictions:
loaded_model = pickle.load(open('trained_model.sav', 'rb'))

X_new = X_test[200]
#"print(X_new)
print("predicted value:", Y_test[200])

prediction = loaded_model.predict(X_new)
print("actual value: ",prediction[0])

if(prediction[0] == 0):
    print("Negative tweet")
else:
    print("Positive tweet")

predicted value: 1
actual value:  1
Positive tweet
